# **Environment Setup**

_In this section, we pin our dependencies (NumPy, Captum, etc.) and verify that we are running on the correct framework versions.





In [ ]:
# ─── Cell 1: INSTALL CUDA-ENABLED PYTORCH ────────────────────────────────────────
# (You can remove the heavy uninstall/rm -rf; a fresh Colab VM comes clean.)

# 1) Install matching CUDA PyTorch and TorchVision
!pip install --quiet \
    --index-url https://download.pytorch.org/whl/cu121 \
    torch==2.2.0+cu121 torchvision==0.17.0+cu121

# 2) Install Captum & SciPy
!pip install --quiet captum scipy


In [ ]:

# ─── End of installs ─────────────────────────────────────────────────────────────

import torch, torchvision
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# 1. Device & seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

# 2. Load model + transforms
from torchvision.models import resnet50, ResNet50_Weights
weights    = ResNet50_Weights.DEFAULT
model      = resnet50(weights=weights).to(device).eval()
preprocess = weights.transforms()

# 3. Confirm
print("Using device:", device)
print("  torch version:", torch.__version__)
print("  torchvision:", torchvision.__version__)
print("  numpy version:", np.__version__)


In [ ]:
# ─── Cell 1.1: Install & import LIME ────────────────────────────
!pip install --quiet lime

from lime import lime_image
from skimage.segmentation import slic
from torchvision.transforms import ToPILImage
import numpy as np
import torch
import cv2
from lime import lime_image


In [ ]:
import torch
print("Using device:", device)
print("CUDA available?", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "—")


### Cell: Load Pascal VOC 2007 Validation Set

This cell downloads and prepares the VOC 2007 validation split, loads a pre-trained ResNet50 (with its recommended preprocessing), and builds a simple in-memory list called `dataset`. Each entry in `dataset` is a tuple  (PIL_Image, [list_of_gt_boxes])` containing the raw image and its ground‐truth bounding boxes. After running, you can use `dataset` directly for downstream evaluation.



In [ ]:
# ─── Cell 2: Load Pascal VOC 2007 val set ──────────────────────────────────────────
from torchvision.datasets import VOCDetection
from pathlib import Path

# (model, device, and preprocess were set in Cell 1; no need to reload them here)

# Download & prepare VOC 2007 validation set
voc_val = VOCDetection(
    root="./data",
    year="2007",
    image_set="val",
    download=True,
    transform=None       # keep raw PIL.Image so we can do custom BGR/transform later
)

# Build our (PIL-image, [list of GT boxes]) pairs
dataset = []
for img_pil, target in voc_val:
    boxes = []
    objs = target["annotation"].get("object", [])
    if isinstance(objs, dict):
        objs = [objs]
    for obj in objs:
        bb = obj["bndbox"]
        boxes.append([
            int(bb["xmin"]), int(bb["ymin"]),
            int(bb["xmax"]), int(bb["ymax"])
        ])
    dataset.append((img_pil, boxes))

print(f"VOC dataset ready with {len(dataset)} images")


### **Cell: Helper Functions for CAM, Saliency, and Integrated Gradients**

Defines three utility functions:

1. **`compute_cam(...)`**  
   - Registers hooks on the chosen ResNet50 layer, runs a forward/backward pass, and builds a Grad-CAM heatmap plus overlay for a given BGR image.

2. **`compute_saliency(...)`**  
   - Runs a vanilla gradient backpropagation to compute a saliency map  (absolute gradient max over channels), then normalizes and resizes it to the input image size.

3. **`compute_ig(...)`**  
   - Uses Captum’s IntegratedGradients to compute per-pixel attributions collapses channels (absolute sum), normalizes, and resizes back to the original image dimensions.

All three functions return a single‐channel heatmap (shape H×W) that can be used for further evaluation or visualization.


In [ ]:
from PIL import Image

def compute_cam(img_bgr, model, preprocess, target_layer=None, device=device):
    """Returns (heatmap, overlay) for a single BGR image."""
    # 1) Hook storage
    activations, gradients = {}, {}

    if target_layer is None:
        target_layer = model.layer4[-1]

    def fwd_hook(mod, inp, out):    activations['feat'] = out.detach()
    def bwd_hook(mod, grad_in, grad_out): gradients['grad'] = grad_out[0].detach()

    h_f = target_layer.register_forward_hook(fwd_hook)
    h_b = target_layer.register_backward_hook(bwd_hook)

    # 2) Preprocess
    rgb    = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    inp = preprocess(pil).unsqueeze(0).to(device)

    # 3) Forward/backward
    out = model(inp)
    cls = out.argmax(dim=1).item()
    model.zero_grad()
    out[0, cls].backward()

    # 4) Build CAM
    feat = activations['feat'][0].cpu().numpy()      # C×H×W
    grad = gradients['grad'][0].cpu().numpy()       # C×H×W
    weights_ = grad.mean(axis=(1,2))                # C
    cam = np.zeros(feat.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights_):
        cam += w * feat[i]
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (img_bgr.shape[1], img_bgr.shape[0]))
    heatmap = (cam - cam.min())/(cam.max()-cam.min()+1e-8)

    # 5) Overlay
    colored = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(colored, 0.5, img_bgr, 0.5, 0)

    # 6) Clean up hooks
    h_f.remove(); h_b.remove()
    return heatmap, overlay

In [ ]:
# ── Cell: compare first block vs last block of ResNet ──────────────────────────
import cv2
import numpy as np
from PIL import Image
import torch # Import torch as it's used in compute_cam
from torchvision.models import resnet50, ResNet50_Weights # Import these as they are used

early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

# Load an image from the dataset
img_pil, _ = dataset[0] # Assuming dataset is a list of (image, boxes) tuples.

# Convert PIL image to OpenCV BGR image
img_bgr = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

heat_early, overlay_early = compute_cam(img_bgr, model, preprocess,
                                       target_layer=early_layer,
                                       device=device)
heat_late,  overlay_late  = compute_cam(img_bgr, model, preprocess,
                                       target_layer=late_layer,
                                       device=device)

In [ ]:
def compute_saliency(img_bgr, model, preprocess, device=device):
    """Vanilla gradient saliency, resized to the original image size."""
    model.zero_grad()
    # 1) Preprocess to model input size
    rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    inp = preprocess(pil).unsqueeze(0).to(device)
    inp.requires_grad_(True)

    # 2) Forward
    model.zero_grad()
    out = model(inp)
    cls = out.argmax(dim=1).item()

    # 3) Backward on the *scalar* score for that class
    score = out[0, cls]
    score.backward()

    # 4) Extract gradient w.r.t. input, sum channels → H×W
    grad = inp.grad.abs()[0].cpu().numpy()     # shape: 3×h×w
    sal = grad.max(axis=0)                     # shape: h×w

    # 4) Normalize
    sal = (sal - sal.min())/(sal.max() - sal.min() + 1e-8)

    # 5) Normalize & resize back to original
    sal = (sal - sal.min()) / (sal.max() - sal.min() + 1e-8)
    H, W = img_bgr.shape[:2]
    sal = cv2.resize(sal, (W, H))

    return sal


In [ ]:
# Cell 8: redefine compute_ig to accept a target_layer
from captum.attr import IntegratedGradients, LayerIntegratedGradients
import torch
import numpy as np
import cv2
from PIL import Image

def compute_ig(
    img_bgr,           # H×W×3 OpenCV BGR array
    model,             # pretrained model
    preprocess,        # torchvision transforms for the model
    device,            # "cuda" or "cpu"
    baseline=None,     # baseline tensor, default zero
    n_steps=50,        # number of IG steps
    target_layer=None  # if None: input‐IG; else: layer‐IG at this module
):
    """Returns a [0,1] attribution map resized to the original H×W."""
    # 1) Preprocess
    pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    inp = preprocess(pil).unsqueeze(0).to(device)
    if baseline is None:
        baseline = torch.zeros_like(inp).to(device)

    # 2) Predicted class
    with torch.no_grad():
        logits = model(inp)
    cls = logits.argmax(dim=1).item()

    # 3) Choose Captum explainer
    if target_layer is None:
        explainer = IntegratedGradients(model)
        attributions = explainer.attribute(
            inp,
            baselines=baseline,
            target=cls,
            n_steps=n_steps
        )  # shape (1,3,H,W)
        # collapse to H×W
        ig_map = attributions.squeeze().abs().sum(dim=0).cpu().numpy()
    else:
        explainer = LayerIntegratedGradients(model, target_layer)
        attributions = explainer.attribute(
            inp,
            baselines=baseline,
            target=cls,
            n_steps=n_steps
        )  # shape (1,C_l,H_l,W_l)
        # collapse channels, average over channels
        ig_map = attributions.squeeze().abs().mean(dim=0).cpu().numpy()

    # 4) Normalize & resize
    ig_map = (ig_map - ig_map.min())/(ig_map.max()-ig_map.min()+1e-8)
    H, W = img_bgr.shape[:2]
    ig_map = cv2.resize(ig_map, (W, H))

    return ig_map


In [ ]:
# Cell 9: compare IG at early vs late ResNet layers
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

ig_map_early = compute_ig(img_bgr, model, preprocess, device=device,
                          baseline=None, n_steps=50,
                          target_layer=early_layer)
ig_map_late  = compute_ig(img_bgr, model, preprocess, device=device,
                          baseline=None, n_steps=50,
                          target_layer=late_layer)


### Cell: Insertion/Deletion AUC and IoU Scoring

**`insertion_deletion(model, preprocess, img_bgr, heatmap, steps=50)`:**  
1. Converts the input BGR image to a PIL image and creates a blurred version.  
2. Flattens the heatmap to rank pixels by attribution strength.  
3. Iteratively “inserts” top‐attributed pixels into the blurred image and records the model’s softmax score for the predicted class (`ins_scores`).  
4. Iteratively “deletes” top‐attributed pixels from the original image (replacing them with blurred pixels) and records the model’s softmax score (`del_scores`).  
5. Returns two lists: insertion AUC scores (`ins_scores`) and deletion AUC scores (`del_scores`).

In [ ]:
def insertion_deletion(model, preprocess, img_bgr, heatmap, steps=50):
    import numpy as np, cv2
    from PIL import Image

    # 1) Prepare original & blurred
    rgb      = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    inp = preprocess(pil).unsqueeze(0).to(device)

    with torch.no_grad():
        orig_out = model(inp)
    cls      = orig_out.argmax(dim=1).item()
    blur     = cv2.GaussianBlur(img_bgr, (51,51), 0)

    # 2) Pixel ordering
    h, w     = heatmap.shape
    flat_idx = np.argsort(heatmap.flatten())[::-1]
    total    = h * w

    ins_scores, del_scores = [], []
    for i in np.linspace(0, total, steps, endpoint=False, dtype=int):
        mask = np.zeros(total, dtype=bool)
        mask[flat_idx[:i]] = True
        mask = mask.reshape(h, w)

        # insertion
        merged = blur.copy(); merged[mask] = img_bgr[mask]
        pil_m  = Image.fromarray(cv2.cvtColor(merged, cv2.COLOR_BGR2RGB))
        inp_i  = preprocess(pil_m).unsqueeze(0).to(device)
        with torch.no_grad():
            ins_scores.append(model(inp_i).softmax(1)[0, cls].item())

        # deletion
        deleted = img_bgr.copy(); deleted[mask] = blur[mask]
        pil_d   = Image.fromarray(cv2.cvtColor(deleted, cv2.COLOR_BGR2RGB))
        inp_d   = preprocess(pil_d).unsqueeze(0).to(device)
        with torch.no_grad():
            del_scores.append(model(inp_d).softmax(1)[0, cls].item())

    return ins_scores, del_scores


**`iou_score(pred_mask, true_mask)`:**  
Computes Intersection over Union (IoU) between a binary `pred_mask` (from thresholded heatmap) and `true_mask` (ground‐truth bounding regions).  
- **Intersection:** `np.logical_and(pred_mask, true_mask).sum()`  
- **Union:** `np.logical_or(pred_mask, true_mask).sum()`  
- Returns `intersection / (union + 1e-8)` to avoid division by zero.

These helper functions allow us to quantitatively evaluate how well each attribution method localizes the ground‐truth object (IoU) and how the predicted confidence changes as we insert or delete pixels (Insertion/Deletion AUC).  

In [ ]:
def iou_score(pred_mask, true_mask):
    inter = np.logical_and(pred_mask, true_mask).sum()
    uni   = np.logical_or(pred_mask, true_mask).sum()
    return inter/(uni+1e-8)


In [ ]:
from captum.attr import IntegratedGradients

# Assuming `model`, `device`, `preprocess` already exist
ig_explainer = IntegratedGradients(model)

def compute_ig_map(
    input_tensor,        # 4D tensor (1,C,H,W) already on device
    target_class,        # int
    original_shape,      # (H, W)
    baseline=None,
    n_steps=50
):
    if baseline is None:
        baseline = torch.zeros_like(input_tensor)
    attributions, _ = ig_explainer.attribute(
        input_tensor,
        baselines=baseline,
        target=target_class,
        n_steps=n_steps,
        return_convergence_delta=True
    )
    sal = attributions.squeeze(0).abs().sum(dim=0)
    sal = (sal - sal.min())/(sal.max()-sal.min()+1e-8)
    sal_np = sal.detach().cpu().numpy()
    H, W = original_shape
    return cv2.resize(sal_np, (W, H))


In [ ]:
# ─── Cell 2.1 (fixed): Define compute_lime_map ────────────────────────
explainer = lime_image.LimeImageExplainer()

def compute_lime_map(img_bgr, model, preprocess, device,
                     num_samples=500,
                     segmentation_kwargs={"n_segments":50, "compactness":10}):
    import numpy as np, cv2, torch
    from skimage.segmentation import slic
    from torchvision.transforms import ToPILImage

    # 1) BGR→RGB
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # 2) LIME requires a batch‐predict fn
    def batch_predict(images):
        batch = torch.stack([
            preprocess(ToPILImage()(im)).to(device)
            for im in images
        ], dim=0)
        with torch.no_grad():
            probs = torch.softmax(model(batch), dim=1)
        return probs.cpu().numpy()

    # 3) Run LIME
    lime_exp = explainer.explain_instance(
        image=img_rgb,
        classifier_fn=batch_predict,
        top_labels=1,
        num_samples=num_samples,
        segmentation_fn=lambda x: slic(x, **segmentation_kwargs)
    )

    # 4) Correct unpack: first is the image, second is the H×W mask
    _, seg_mask = lime_exp.get_image_and_mask(
        lime_exp.top_labels[0],
        positive_only=True,
        num_features=segmentation_kwargs["n_segments"],
        hide_rest=False
    )

    # 5) Build continuous heatmap from segment weights
    weights = dict(lime_exp.local_exp[lime_exp.top_labels[0]])
    heatmap = np.zeros_like(seg_mask, dtype=float)
    for seg_id, w in weights.items():
        heatmap[seg_mask == seg_id] = w

    # 6) Normalize to [0,1]
    heatmap = np.clip(heatmap, 0, None)
    heatmap = (heatmap - heatmap.min())/(heatmap.max()-heatmap.min()+1e-8)

    return heatmap


## Evaluating Grad-CAM IoU (and Optional Insertion/Deletion) on VOC 2007

The cell below iterates over the first 200 images in the Pascal VOC 2007 validation set (stored in `dataset`) and computes:

1. **Grad-CAM heatmap** for each image using your pretrained ResNet50 model and the helper function `compute_cam(...)`.  
2. **Binary mask** by thresholding the Grad-CAM heatmap at the 80th percentile (`best_p = 80`).  
3. **IoU score** between this binary mask and the ground-truth bounding boxes (`iou_score(...)`), accumulating results into `cam_iou_list`.  
4. *(Optional)* **Insertion/Deletion AUC** metrics for each Grad-CAM mask by calling `insertion_deletion(...)` with a coarser step size (10) and storing results in `cam_ins_list` and `cam_del_list`.

**Prerequisites** (already defined in earlier cells):  
- `model`, `device`, and `preprocess` (ResNet50 + its transforms)  
- `dataset` (list of `(PIL.Image, [gt_boxes])` tuples)  
- Helper functions: `compute_cam`, `iou_score`, `insertion_deletion`  

After running this cell, you’ll have:  
- `cam_iou_list`: IoU values (`mean ± std`) for the first 200 validation images  
- *(Optional)* `cam_ins_list` and `cam_del_list`: insertion and deletion AUC scores for Grad-CAM masks  


In [ ]:
# ── Cell A: Grad-CAM IoU + (optional) Insertion/Deletion on first 200 VOC images, early vs. late layers ──
import time
import numpy as np
import torch
import cv2
from PIL import Image
from pathlib import Path

# (Assumes `model`, `preprocess`, `dataset`, `compute_cam`, `iou_score`, and `insertion_deletion` are in scope.)

# 1) Hyperparameters
best_p     = 80      # percentile threshold to binarize a CAM heatmap
max_images = 200     # how many images to process
idel_steps = 10      # (coarser) steps for insertion/deletion

# 2) Define early & late layers
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

# 3) Storage lists for both layers
cam_iou_early = []
cam_ins_early = []
cam_del_early = []

cam_iou_late  = []
cam_ins_late  = []
cam_del_late  = []

# 4) Loop
start_time = time.time()
for idx, (img, gt_boxes) in enumerate(dataset):
    if idx >= max_images:
        break

    # Convert any type to BGR uint8
    if isinstance(img, (str, Path)):
        img_pil = Image.open(img).convert("RGB")
        img_np  = np.array(img_pil, dtype=np.uint8)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    elif isinstance(img, Image.Image):
        img_np  = np.array(img.convert("RGB"), dtype=np.uint8)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    elif isinstance(img, torch.Tensor):
        arr = img.detach().cpu()
        if arr.ndim == 4 and arr.shape[0] == 1:
            arr = arr[0]
        arr = (arr * 255.0).clamp(0,255).byte()
        arr = arr.permute(1,2,0).cpu().numpy()
        img_bgr = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
    else:
        raise RuntimeError(f"Unrecognized image type: {type(img)}")
    H, W = img_bgr.shape[:2]

    # — compute CAM & metrics for early layer —
    cam_map_e, _  = compute_cam(img_bgr, model, preprocess,
                                target_layer=early_layer, device=device)
    thr_e        = np.percentile(cam_map_e, best_p)
    mask_e       = (cam_map_e >= thr_e).astype(np.uint8)
    true_mask    = np.zeros_like(mask_e)
    for x1,y1,x2,y2 in gt_boxes:
        true_mask[y1:y2, x1:x2] = 1
    cam_iou_early.append(iou_score(mask_e, true_mask))
    ins_e, del_e = insertion_deletion(model, preprocess, img_bgr, cam_map_e, steps=idel_steps)
    cam_ins_early.append(np.trapz(ins_e) / len(ins_e))
    cam_del_early.append(np.trapz(1.0 - np.array(del_e)) / len(del_e))

    # — compute CAM & metrics for late layer —
    cam_map_l, _  = compute_cam(img_bgr, model, preprocess,
                                target_layer=late_layer, device=device)
    thr_l        = np.percentile(cam_map_l, best_p)
    mask_l       = (cam_map_l >= thr_l).astype(np.uint8)
    true_mask    = np.zeros_like(mask_l)
    for x1,y1,x2,y2 in gt_boxes:
        true_mask[y1:y2, x1:x2] = 1
    cam_iou_late.append(iou_score(mask_l, true_mask))
    ins_l, del_l = insertion_deletion(model, preprocess, img_bgr, cam_map_l, steps=idel_steps)
    cam_ins_late.append(np.trapz(ins_l) / len(ins_l))
    cam_del_late.append(np.trapz(1.0 - np.array(del_l)) / len(del_l))

# ── End loop ──
elapsed = time.time() - start_time

# 5) Print results
print(f"Processed {len(cam_iou_early)} images in {elapsed:.1f}s\n")

print("EARLY LAYER (layer1[0])")
print(f" IoU   = {np.mean(cam_iou_early):.3f} ± {np.std(cam_iou_early):.3f}")
print(f" Ins AUC = {np.mean(cam_ins_early):.3f} ± {np.std(cam_ins_early):.3f}")
print(f" Del AUC = {np.mean(cam_del_early):.3f} ± {np.std(cam_del_early):.3f}\n")

print("LATE LAYER (layer4[-1])")
print(f" IoU   = {np.mean(cam_iou_late):.3f} ± {np.std(cam_iou_late):.3f}")
print(f" Ins AUC = {np.mean(cam_ins_late):.3f} ± {np.std(cam_ins_late):.3f}")
print(f" Del AUC = {np.mean(cam_del_late):.3f} ± {np.std(cam_del_late):.3f}")


In [ ]:
# ─── Cell A.1: Grad-CAM on First 400 VOC Images ──────────────────────────────
import time, numpy as np
from pathlib import Path
import torch, cv2
from PIL import Image

# Hyperparameters
best_p          = 80      # percentile for binarization
idel_steps      = 10      # insertion/deletion steps
max_images_full = 400     # process up to 400 images

# Layers you compared in Cell A
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

# Storage
gc_iou_e, gc_ins_e, gc_del_e = [], [], []
gc_iou_l, gc_ins_l, gc_del_l = [], [], []

start = time.time()
for idx, (img, gt_boxes) in enumerate(dataset):
    if idx >= max_images_full:
        break

    # — Load into PIL.Image —
    if isinstance(img, (str, Path)):
        img_pil = Image.open(img).convert("RGB")
    elif isinstance(img, Image.Image):
        img_pil = img.convert("RGB")
    else:
        t = img.detach().cpu()
        if t.ndim == 4 and t.size(0)==1:
            t = t.squeeze(0)
        arr = (t.permute(1,2,0).clamp(0,1)*255).byte().numpy()
        img_pil = Image.fromarray(arr)

    # — Preprocess & predict class —
    inp = preprocess(img_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        cls = int(model(inp).argmax(dim=1).item())

    # — Grad-CAM at early layer —
    # Replace grad_cam with compute_cam
    cam_e, _ = compute_cam(cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR), model, preprocess, target_layer=early_layer, device=device)  # H×W float map
    thr_e = np.percentile(cam_e, best_p)
    mask_e = (cam_e >= thr_e).astype(np.uint8)
    # build GT mask
    true_e = np.zeros_like(mask_e)
    H, W = mask_e.shape
    for x1,y1,x2,y2 in gt_boxes:
        true_e[y1:y2, x1:x2] = 1

    gc_iou_e.append(iou_score(mask_e, true_e))
    # insertion_deletion expects img_bgr and heatmap. Use img_bgr and cam_e
    ins_e, del_e = insertion_deletion(model, preprocess, img_bgr=cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR), heatmap=cam_e, steps=idel_steps)
    gc_ins_e.append(np.trapz(ins_e)/len(ins_e))
    gc_del_e.append(np.trapz(1-np.array(del_e))/len(del_e))

    # — Grad-CAM at late layer —
    # Replace grad_cam with compute_cam
    cam_l, _ = compute_cam(cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR), model, preprocess, target_layer=late_layer, device=device)
    thr_l = np.percentile(cam_l, best_p)
    mask_l = (cam_l >= thr_l).astype(np.uint8)
    true_l = np.zeros_like(mask_l)
    for x1,y1,x2,y2 in gt_boxes:
        true_l[y1:y2, x1:x2] = 1

    gc_iou_l.append(iou_score(mask_l, true_l))
    # insertion_deletion expects img_bgr and heatmap. Use img_bgr and cam_l
    ins_l, del_l = insertion_deletion(model, preprocess, img_bgr=cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR), heatmap=cam_l, steps=idel_steps)
    gc_ins_l.append(np.trapz(ins_l)/len(ins_l))
    gc_del_l.append(np.trapz(1-np.array(del_l))/len(del_l))

elapsed = time.time() - start

print(f"Expanded Grad-CAM on {len(gc_iou_e)} images × 400 cap in {elapsed:.1f}s\n")
print("EARLY layer:")
print(f" IoU   = {np.mean(gc_iou_e):.3f} ± {np.std(gc_iou_e):.3f}")
print(f" Ins   = {np.mean(gc_ins_e):.3f} ± {np.std(gc_ins_e):.3f}")
print(f" Del   = {np.mean(gc_del_e):.3f} ± {np.std(gc_del_e):.3f}\n")
print("LATE layer:")
print(f" IoU   = {np.mean(gc_iou_l):.3f} ± {np.std(gc_iou_l):.3f}")
print(f" Ins   = {np.mean(gc_ins_l):.3f} ± {np.std(gc_ins_l):.3f}")
print(f" Del   = {np.mean(gc_del_l):.3f} ± {np.std(gc_del_l):.3f}")

## Evaluating Saliency IoU (and Optional Insertion/Deletion) on VOC 2007

This cell processes the first `max_images` from the Pascal VOC 2007 validation set and computes:

1. **Saliency Map** for each image using the pretrained ResNet50 model and the helper function `compute_saliency(...)`.  
2. **Binary Mask** by thresholding the saliency map at the `best_p`-th percentile (`best_p = 80`).  
3. **IoU Score** between that binary mask and the ground-truth bounding boxes (`iou_score(...)`), stored in `sal_iou_list`.  
4. *(Optional)* **Insertion/Deletion AUC** metrics for each saliency mask via `insertion_deletion(...)` (using a coarser step size `idel_steps = 10`), stored in `sal_ins_list` and `sal_del_list`.

**Assumes (defined in earlier cells):**  
- `model`, `device`, `preprocess` (ResNet50 + its image transforms)  
- `dataset` (list of `(PIL.Image, [gt_boxes])` tuples)  
- Helper functions:  
  - `compute_saliency(img_bgr, model, preprocess, device)` → returns a normalized saliency heatmap  
  - `iou_score(pred_mask, true_mask)` → computes intersection-over-union  
  - `insertion_deletion(model, preprocess, img_bgr, heatmap, steps)` → returns insertion/deletion curves  

After running this cell, you will have:  
- `sal_iou_list`: IoU values (`mean ± std`) for the first `max_images` images  
- *(Optional)* `sal_ins_list` / `sal_del_list`: insertion and deletion AUC scores for saliency  



In [ ]:
# ── Cell B: Saliency IoU + Insertion/Deletion on first max_images VOC images ──────
import time
import numpy as np
import torch
import cv2
from PIL import Image
from pathlib import Path

# (We assume the following variables/functions are already defined in previous cells:)
#   - model            : a ResNet50 (or similar) in .eval() mode, on the proper device
#   - preprocess       : the torchvision.transforms() for ResNet50
#   - dataset          : list/iterable of (PILImage or Tensor or str/Path, gt_boxes) tuples
#   - compute_saliency : function(img_bgr, model, preprocess, device) → H×W saliency map (0–1 float)
#   - insertion_deletion(model, preprocess, img_bgr, heatmap, steps) → (ins_scores, del_scores)
#   - iou_score        : function(pred_mask, true_mask) → intersection_over_union float
#   - device           : "cuda" or "cpu"

# 1) Hyperparameters
best_p     = 80    # percentile threshold for turning a saliency map into a binary mask
max_images = 200   # process at most this many images
idel_steps = 10    # # of insertion/deletion steps (coarser = faster)

# 2) Prepare storage
sal_iou_list = []
sal_ins_list = []
sal_del_list = []

# 3) Loop over first `max_images` entries
start_time = time.time()

for idx, (img, gt_boxes) in enumerate(dataset):
    if idx >= max_images:
        break

    # ── 3.1) Convert whatever `img` is into a PIL.Image in "RGB" mode ───────────────
    if isinstance(img, (str, Path)):
        # If `img` is a filesystem path, open it with PIL and convert to RGB
        img_pil = Image.open(img).convert("RGB")
    elif isinstance(img, Image.Image):
        # Already a PIL.Image; just ensure it's RGB
        img_pil = img.convert("RGB")
    elif isinstance(img, torch.Tensor):
        # If it's a torch.Tensor, assume shape = (C,H,W) or (1,C,H,W)
        t = img.detach().cpu()
        if t.ndim == 4 and t.size(0) == 1:
            t = t.squeeze(0)
        # Now t is (C,H,W); permute → (H,W,C)
        t = t.permute(1, 2, 0)
        # If values are normalized floats (0–1), clamp&scale to [0–255]
        arr = (t.clamp(0, 1) * 255.0).byte().numpy()
        img_pil = Image.fromarray(arr)
    else:
        raise RuntimeError(f"Unrecognized image type at index {idx}: {type(img)}")

    # ── 3.2) Convert PIL.Image → NumPy(RGB) → BGR for OpenCV ─────────────────────────
    img_np = np.array(img_pil, dtype=np.uint8)             # shape = (H, W, 3), RGB order
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)      # now shape = (H, W, 3), BGR order
    H, W = img_bgr.shape[:2]

    # ── 3.3) Compute Saliency Map using your helper function ─────────────────────────
    sal_map = compute_saliency(img_bgr, model, preprocess, device)  # float32 H×W, values ∈ [0,1]

    # ── 3.4) Threshold at the `best_p`‐percentile to get a binary mask ─────────────
    thr = np.percentile(sal_map, best_p)
    sal_mask = (sal_map >= thr).astype(np.uint8)  # H×W mask, 0/1

    # ── 3.5) Build “true_mask” from GT boxes (list of [x1,y1,x2,y2]) ────────────────
    true_mask = np.zeros_like(sal_mask, dtype=np.uint8)
    for (x1, y1, x2, y2) in gt_boxes:
        xx1 = max(0, min(W, x1))
        yy1 = max(0, min(H, y1))
        xx2 = max(0, min(W, x2))
        yy2 = max(0, min(H, y2))
        true_mask[yy1:yy2, xx1:xx2] = 1

    # ── 3.6) Compute IoU between saliency‐mask and GT‐mask ─────────────────────────────
    sal_iou_list.append(iou_score(sal_mask, true_mask))

    # ── 3.7) Insertion/Deletion AUC (coarse, for speed) ─────────────────────────────
    ins_vals, del_vals = insertion_deletion(model, preprocess, img_bgr, sal_map, steps=idel_steps)
    sal_ins_list.append(np.trapz(ins_vals) / len(ins_vals))
    sal_del_list.append(np.trapz(1.0 - np.array(del_vals)) / len(del_vals))

elapsed = time.time() - start_time

# 4) Print summary
print(f"Saliency: processed {len(sal_iou_list)} images in {elapsed:.1f} seconds")
print(f"├─ Saliency IoU   = {np.mean(sal_iou_list):.3f} ± {np.std(sal_iou_list):.3f}")
print(f"├─ Saliency Ins AUC = {np.mean(sal_ins_list):.3f} ± {np.std(sal_ins_list):.3f}")
print(f"└─ Saliency Del AUC = {np.mean(sal_del_list):.3f} ± {np.std(sal_del_list):.3f}")


In [ ]:
# ─── Cell B.3: Saliency on First 400 VOC Images ──────────────────────────────
import time, numpy as np
from pathlib import Path
import torch, cv2
from PIL import Image

# Hyperparameters
best_p          = 80      # percentile threshold
idel_steps      = 10      # insertion/deletion steps
max_images_full = 400     # cap at 400 images

# Storage
sal_iou_list, sal_ins_list, sal_del_list = [], [], []

start = time.time()
for idx, (img, gt_boxes) in enumerate(dataset):
    if idx >= max_images_full:
        break

    # — Load as BGR for compute_saliency() —
    if isinstance(img, (str, Path)):
        img_pil = Image.open(img).convert("RGB")
    elif isinstance(img, Image.Image):
        img_pil = img.convert("RGB")
    else:
        t = img.detach().cpu()
        if t.ndim == 4 and t.size(0)==1:
            t = t.squeeze(0)
        arr = (t.permute(1,2,0).clamp(0,1)*255).byte().numpy()
        img_pil = Image.fromarray(arr)
    img_np  = np.array(img_pil, np.uint8)
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    # — Compute vanilla saliency map —
    sal_map = compute_saliency(img_bgr, model, preprocess, device)

    # — Binarize →
    thr       = np.percentile(sal_map, best_p)
    sal_mask  = (sal_map >= thr).astype(np.uint8)

    # — Build GT mask →
    true_mask = np.zeros_like(sal_mask)
    H, W = sal_mask.shape
    for x1,y1,x2,y2 in gt_boxes:
        true_mask[y1:y2, x1:x2] = 1

    # — Metrics →
    sal_iou_list.append(iou_score(sal_mask, true_mask))
    ins_vals, del_vals = insertion_deletion(model, preprocess, img_bgr, sal_map, steps=idel_steps)
    sal_ins_list.append(np.trapz(ins_vals)/len(ins_vals))
    sal_del_list.append(np.trapz(1-np.array(del_vals))/len(del_vals))

elapsed = time.time() - start

print(f"Expanded Saliency on {len(sal_iou_list)} images × 400 cap in {elapsed:.1f}s\n")
print(f" IoU   = {np.mean(sal_iou_list):.3f} ± {np.std(sal_iou_list):.3f}")
print(f" Ins   = {np.mean(sal_ins_list):.3f} ± {np.std(sal_ins_list):.3f}")
print(f" Del   = {np.mean(sal_del_list):.3f} ± {np.std(sal_del_list):.3f}")


In [ ]:
# ─── Cell B.1: SmoothGrad Wrapper for Your Saliency ───
import numpy as np

def compute_smoothgrad_saliency(img_bgr, model, preprocess, device,
                                n_samples=25, noise_sigma=0.1):
    """
    A SmoothGrad variant that calls your compute_saliency on noisy inputs.
    Returns the average saliency map (shape H×W, values in [0,1]).
    """
    grads = []
    for _ in range(n_samples):
        # 1) Add Gaussian noise in pixel space
        noise = np.random.normal(0, noise_sigma, img_bgr.shape).astype(np.float32)
        noisy_bgr = np.clip(img_bgr.astype(np.float32) + noise, 0, 255).astype(np.uint8)
        # 2) Compute vanilla saliency on the noisy image
        grads.append(compute_saliency(noisy_bgr, model, preprocess, device))
    # 3) Mean of all saliency maps
    return np.mean(grads, axis=0)


In [ ]:
# ─── Cell B.2: SmoothGrad IoU & Insertion/Deletion Sweep ───
import matplotlib.pyplot as plt

noise_levels = [0.05, 0.1, 0.2]
sg_iou_means = []
sg_ins_means = []
sg_del_means = []

for sigma in noise_levels:
    iou_list, ins_list, del_list = [], [], []
    for idx, (img, gt_boxes) in enumerate(dataset):
        if idx >= max_images_full:  # reuse your max_images setting
            break

        # — convert to BGR as before —
        img_pil = (Image.open(img).convert("RGB")
                   if isinstance(img, (str, Path))
                   else (img.convert("RGB") if isinstance(img, Image.Image)
                         else None))
        img_np = np.array(img_pil, dtype=np.uint8)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        # 1) SmoothGrad saliency
        sal = compute_smoothgrad_saliency(
            img_bgr, model, preprocess, device,
            n_samples=25, noise_sigma=sigma
        )
        # 2) Threshold → mask
        thr = np.percentile(sal, best_p)
        mask = (sal >= thr).astype(np.uint8)

        # 3) Build true_mask (reuse your box loop)
        H, W = mask.shape
        true_mask = np.zeros_like(mask)
        for (x1,y1,x2,y2) in gt_boxes:
            true_mask[y1:y2, x1:x2] = 1

        # 4) Metrics
        iou_list.append(iou_score(mask, true_mask))
        ins, dels = insertion_deletion(model, preprocess, img_bgr, sal, steps=idel_steps)
        ins_list.append(np.trapz(ins)   / len(ins))
        del_list.append(np.trapz(1.0 - np.array(dels)) / len(dels))

    sg_iou_means.append(np.mean(iou_list))
    sg_ins_means.append(np.mean(ins_list))
    sg_del_means.append(np.mean(del_list))

# 4) Plot results
plt.figure(figsize=(6,4))
plt.plot(noise_levels, sg_iou_means, marker='o', label='IoU')
plt.plot(noise_levels, sg_ins_means, marker='o', label='Insertion AUC')
plt.plot(noise_levels, sg_del_means, marker='o', label='Deletion AUC')
plt.xlabel('SmoothGrad noise σ')
plt.legend()
plt.title('SmoothGrad Sensitivity')
plt.show()


## Evaluating Integrated Gradients IoU (and Optional Insertion/Deletion) on VOC 2007

This cell processes the first `max_images` from the Pascal VOC 2007 validation set and computes:

1. **Integrated Gradients (IG) Attribution Map** for each image using the pretrained ResNet50 model and the helper function `compute_ig_map(...)`.  
2. **Binary Mask** by thresholding the IG map at the `best_p`-th percentile (`best_p = 80`).  
3. **IoU Score** between that binary mask and the ground-truth bounding boxes (`iou_score(...)`), stored in `ig_iou_list`.  
4. *(Optional)* **Insertion/Deletion AUC** metrics for each IG mask via `insertion_deletion(...)` (using a coarser step size `idel_steps = 10`), stored in `ig_ins_list` and `ig_del_list`.

**Assumes (already defined earlier):**  
- `model`, `device`, `preprocess` (ResNet50 + its transforms)  
- `dataset` (list of `(PIL.Image or Tensor or Path, [gt_boxes])` tuples)  
- Helper functions:  
  • `compute_ig_map(input_tensor, target_class, original_shape, baseline=None, n_steps)` → returns a normalized IG heatmap ([0,1]) resized to (H,W)  
  • `iou_score(pred_mask, true_mask)` → computes intersection‐over‐union  
  • `insertion_deletion(model, preprocess, img_bgr, heatmap, steps)` → returns insertion/deletion score lists  

After running this cell, you will have:  
- `ig_iou_list`: IoU values (mean ± std) for the first `max_images` images  
- *(Optional)* `ig_ins_list` and `ig_del_list`: insertion and deletion AUC scores for IG  


In [ ]:
# ── Cell C: IG IoU + Insertion/Deletion on first max_images VOC images (early vs. late layers) ──
import time
import numpy as np
import torch
import cv2
from PIL import Image
from pathlib import Path

# (Assumes `model`, `preprocess`, `dataset`, `compute_ig`, `insertion_deletion`, `iou_score`, and `device` are in scope.)

# 1) Hyperparameters
best_p     = 80    # percentile threshold for binarizing an IG map
max_images = 200   # max images to process
idel_steps = 10    # insertion/deletion steps
ig_steps   = 50    # IG steps

# 2) Early & late layers for layer‐IG
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

# 3) Storage
ig_iou_early = []; ig_ins_early = []; ig_del_early = []
ig_iou_late  = []; ig_ins_late  = []; ig_del_late  = []

# 4) Loop over dataset
start_time = time.time()
for idx, (img, gt_boxes) in enumerate(dataset):
    if idx >= max_images:
        break

    # 4.1) Load image as PIL
    if isinstance(img, (str, Path)):
        img_pil = Image.open(img).convert("RGB")
    elif isinstance(img, Image.Image):
        img_pil = img.convert("RGB")
    elif isinstance(img, torch.Tensor):
        t = img.detach().cpu()
        if t.ndim == 4 and t.size(0) == 1:
            t = t.squeeze(0)
        arr = (t.permute(1,2,0).clamp(0,1) * 255).byte().numpy()
        img_pil = Image.fromarray(arr)
    else:
        raise RuntimeError(f"Unrecognized image type: {type(img)}")

    # 4.2) To BGR for insertion/deletion
    img_np  = np.array(img_pil, dtype=np.uint8)
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    H, W    = img_bgr.shape[:2]

    # 4.3) Preprocess & predict class
    inp = preprocess(img_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        cls = int(model(inp).argmax(dim=1).item())

    # 4.4) Compute IG at early layer
    ig_map_e = compute_ig(
        img_bgr, model, preprocess, device=device,
        baseline=None, n_steps=ig_steps,
        target_layer=early_layer
    )
    thr_e   = np.percentile(ig_map_e, best_p)
    mask_e  = (ig_map_e >= thr_e).astype(np.uint8)
    true_m  = np.zeros_like(mask_e)
    for x1,y1,x2,y2 in gt_boxes:
        true_m[y1:y2, x1:x2] = 1
    ig_iou_early.append(iou_score(mask_e, true_m))
    ins_e, del_e = insertion_deletion(model, preprocess, img_bgr, ig_map_e, steps=idel_steps)
    ig_ins_early.append(np.trapz(ins_e) / len(ins_e))
    ig_del_early.append(np.trapz(1.0 - np.array(del_e)) / len(del_e))

    # 4.5) Compute IG at late layer
    ig_map_l = compute_ig(
        img_bgr, model, preprocess, device=device,
        baseline=None, n_steps=ig_steps,
        target_layer=late_layer
    )
    thr_l   = np.percentile(ig_map_l, best_p)
    mask_l  = (ig_map_l >= thr_l).astype(np.uint8)
    true_m  = np.zeros_like(mask_l)
    for x1,y1,x2,y2 in gt_boxes:
        true_m[y1:y2, x1:x2] = 1
    ig_iou_late.append(iou_score(mask_l, true_m))
    ins_l, del_l = insertion_deletion(model, preprocess, img_bgr, ig_map_l, steps=idel_steps)
    ig_ins_late.append(np.trapz(ins_l) / len(ins_l))
    ig_del_late.append(np.trapz(1.0 - np.array(del_l)) / len(del_l))

# 5) Summary
elapsed = time.time() - start_time
print(f"Processed {len(ig_iou_early)} images in {elapsed:.1f}s\n")

print("EARLY LAYER (layer1[0])")
print(f" IG IoU   = {np.mean(ig_iou_early):.3f} ± {np.std(ig_iou_early):.3f}")
print(f" Ins AUC  = {np.mean(ig_ins_early):.3f} ± {np.std(ig_ins_early):.3f}")
print(f" Del AUC  = {np.mean(ig_del_early):.3f} ± {np.std(ig_del_early):.3f}\n")

print("LATE LAYER (layer4[-1])")
print(f" IG IoU   = {np.mean(ig_iou_late):.3f} ± {np.std(ig_iou_late):.3f}")
print(f" Ins AUC  = {np.mean(ig_ins_late):.3f} ± {np.std(ig_ins_late):.3f}")
print(f" Del AUC  = {np.mean(ig_del_late):.3f} ± {np.std(ig_del_late):.3f}")


In [ ]:
# ─── Cell C.1: Expanded IG on 400 VOC Images × 100 Steps ───
import time, numpy as np
from pathlib import Path
import torch, cv2
from PIL import Image

# 1) New hyperparameters for the capped run
best_p         = 80              # percentile for thresholding
idel_steps     = 10              # insertion/deletion steps
ig_steps_full  = 100             # IG steps
max_images_full= 400             # cap at 400 images

early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

# 2) Storage for metrics
full_iou_e, full_ins_e, full_del_e = [], [], []
full_iou_l, full_ins_l, full_del_l = [], [], []

# 3) Loop over the first 400 entries in your dataset
start = time.time()
for idx, (img, gt_boxes) in enumerate(dataset):
    if idx >= max_images_full:
        break

    # — load PIL.Image as before —
    if isinstance(img, (str, Path)):
        img_pil = Image.open(img).convert("RGB")
    elif isinstance(img, Image.Image):
        img_pil = img.convert("RGB")
    else:  # tensor or other
        t = img.detach().cpu()
        if t.ndim == 4 and t.size(0)==1:
            t = t.squeeze(0)
        arr = (t.permute(1,2,0).clamp(0,1)*255).byte().numpy()
        img_pil = Image.fromarray(arr)

    # — convert to BGR for ID curves —
    img_np  = np.array(img_pil, np.uint8)
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    # — compute IG at early layer —
    ig_e = compute_ig(
        img_bgr, model, preprocess, device=device,
        baseline=None, n_steps=ig_steps_full,
        target_layer=early_layer
    )
    thr_e  = np.percentile(ig_e, best_p)
    mask_e = (ig_e >= thr_e).astype(np.uint8)
    true_e = np.zeros_like(mask_e)
    for x1,y1,x2,y2 in gt_boxes:
        true_e[y1:y2, x1:x2] = 1
    full_iou_e.append(iou_score(mask_e, true_e))

    ins_e, del_e = insertion_deletion(model, preprocess, img_bgr, ig_e, steps=idel_steps)
    full_ins_e.append(np.trapz(ins_e) / len(ins_e))
    full_del_e.append(np.trapz(1.0 - np.array(del_e)) / len(del_e))

    # — compute IG at late layer —
    ig_l = compute_ig(
        img_bgr, model, preprocess, device=device,
        baseline=None, n_steps=ig_steps_full,
        target_layer=late_layer
    )
    thr_l  = np.percentile(ig_l, best_p)
    mask_l = (ig_l >= thr_l).astype(np.uint8)
    true_l = np.zeros_like(mask_l)
    for x1,y1,x2,y2 in gt_boxes:
        true_l[y1:y2, x1:x2] = 1
    full_iou_l.append(iou_score(mask_l, true_l))

    ins_l, del_l = insertion_deletion(model, preprocess, img_bgr, ig_l, steps=idel_steps)
    full_ins_l.append(np.trapz(ins_l) / len(ins_l))
    full_del_l.append(np.trapz(1.0 - np.array(del_l)) / len(del_l))

# 4) Print summary
elapsed = time.time() - start
print(f"Expanded IG run: {len(full_iou_e)} images × {ig_steps_full} steps in {elapsed:.1f}s\n")

print("EARLY layer:")
print(f" IoU     = {np.mean(full_iou_e):.3f} ± {np.std(full_iou_e):.3f}")
print(f" Ins AUC = {np.mean(full_ins_e):.3f} ± {np.std(full_ins_e):.3f}")
print(f" Del AUC = {np.mean(full_del_e):.3f} ± {np.std(full_del_e):.3f}\n")

print("LATE layer:")
print(f" IoU     = {np.mean(full_iou_l):.3f} ± {np.std(full_iou_l):.3f}")
print(f" Ins AUC = {np.mean(full_ins_l):.3f} ± {np.std(full_ins_l):.3f}")
print(f" Del AUC = {np.mean(full_del_l):.3f} ± {np.std(full_del_l):.3f}")


In [ ]:
# ─── Cell D.1: LIME Pilot on first 200 images ────────────────────
import numpy as np

best_p     = 80
max_images = 200
lime_iou, lime_ins, lime_del = [], [], []

for idx, (img_pil, gt_boxes) in enumerate(dataset):
    if idx >= max_images: break
    rgb     = np.array(img_pil, dtype=np.uint8)
    img_bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

    lm = compute_lime_map(
        img_bgr, model, preprocess, device,
        num_samples=500, segmentation_kwargs={"n_segments":50,"compactness":10}
    )
    thr = np.percentile(lm, best_p)
    mask = (lm >= thr).astype(np.uint8)

    true_m = np.zeros_like(mask)
    for x1,y1,x2,y2 in gt_boxes:
        true_m[y1:y2, x1:x2] = 1

    lime_iou.append(iou_score(mask, true_m))
    ins, dels = insertion_deletion(model, preprocess, img_bgr, lm, steps=idel_steps)
    lime_ins.append(np.trapz(ins)/len(ins))
    lime_del.append(np.trapz(1-np.array(dels))/len(dels))

print(f"LIME Pilot IoU = {np.mean(lime_iou):.3f} ± {np.std(lime_iou):.3f}")


In [ ]:
import numpy as np

print("LIME Pilot Metrics (200 images):")
print(f"  IoU     = {np.mean(lime_iou):.3f} ± {np.std(lime_iou):.3f}")
print(f"  Ins AUC = {np.mean(lime_ins):.3f} ± {np.std(lime_ins):.3f}")
print(f"  Del AUC = {np.mean(lime_del):.3f} ± {np.std(lime_del):.3f}")


In [ ]:
# ─── Cell C.2: LIME Full on first 400 images ─────────────────────
lime_full_iou, lime_full_ins, lime_full_del = [], [], []
max_images_full = 400

for idx, (img_pil, gt_boxes) in enumerate(dataset):
    if idx >= max_images_full: break
    rgb     = np.array(img_pil, dtype=np.uint8)
    img_bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

    lm = compute_lime_map(
        img_bgr, model, preprocess, device,
        num_samples=250, segmentation_kwargs={"n_segments":50,"compactness":10}
    )
    thr  = np.percentile(lm, best_p)
    mask = (lm >= thr).astype(np.uint8)

    true_m = np.zeros_like(mask)
    for x1,y1,x2,y2 in gt_boxes:
        true_m[y1:y2, x1:x2] = 1

    lime_full_iou.append(iou_score(mask, true_m))
    ins, dels = insertion_deletion(model, preprocess, img_bgr, lm, steps=idel_steps)
    lime_full_ins.append(np.trapz(ins)/len(ins))
    lime_full_del.append(np.trapz(1-np.array(dels))/len(dels))

print(f"LIME Full IoU = {np.mean(lime_full_iou):.3f} ± {np.std(lime_full_iou):.3f}")


In [ ]:
import numpy as np

print("LIME Full Metrics (400 images):")
print(f"  IoU     = {np.mean(lime_full_iou):.3f} ± {np.std(lime_full_iou):.3f}")
print(f"  Ins AUC = {np.mean(lime_full_ins):.3f} ± {np.std(lime_full_ins):.3f}")
print(f"  Del AUC = {np.mean(lime_full_del):.3f} ± {np.std(lime_full_del):.3f}")


## Cell D: Consolidate Attribution Metrics into a DataFrame

This cell gathers all of the previously computed metrics—IoU, insertion‐AUC, and deletion‐AUC—for Grad-CAM, Saliency, and Integrated Gradients into a single pandas DataFrame (`df`). Each column corresponds to one metric–method pair:

- **cam_iou**, **cam_ins**, **cam_del**  
- **sal_iou**, **sal_ins**, **sal_del**  
- **ig_iou**,  **ig_ins**,  **ig_del**

By assembling everything into `df`, you can easily preview, summarize, or export all nine metrics at once (e.g., with `df.head()`, `df.describe()`, or saving to CSV).


In [ ]:
import pandas as pd
import numpy as np # Import numpy to use np.min

# 1. Get the minimum length among all lists
# Assuming all lists are defined from previous cell executions
list_lengths = [
    len(cam_iou_early), len(cam_iou_late), len(cam_ins_early), len(cam_ins_late), len(cam_del_early), len(cam_del_late),
    len(sal_iou_list), len(sal_ins_list), len(sal_del_list),
    len(ig_iou_early), len(ig_iou_late), len(ig_ins_early), len(ig_ins_late), len(ig_del_early), len(ig_del_late)
]
min_len = np.min(list_lengths)

print(f"Truncating all lists to the minimum length found: {min_len}")

# 2. Truncate lists to the minimum length
cam_iou_early_truncated = cam_iou_early[:min_len]
cam_iou_late_truncated  = cam_iou_late[:min_len]
cam_ins_early_truncated = cam_ins_early[:min_len]
cam_ins_late_truncated  = cam_ins_late[:min_len]
cam_del_early_truncated = cam_del_early[:min_len]
cam_del_late_truncated  = cam_del_late[:min_len]

sal_iou_list_truncated  = sal_iou_list[:min_len]
sal_ins_list_truncated  = sal_ins_list[:min_len]
sal_del_list_truncated  = sal_del_list[:min_len]

ig_iou_early_truncated  = ig_iou_early[:min_len]
ig_iou_late_truncated   = ig_iou_late[:min_len]
ig_ins_early_truncated  = ig_ins_early[:min_len]
ig_ins_late_truncated   = ig_ins_late[:min_len]
ig_del_early_truncated  = ig_del_early[:min_len]
ig_del_late_truncated   = ig_del_late[:min_len]


# 3. Build pilot DataFrame using truncated lists
df_pilot = pd.DataFrame({
    # Grad-CAM pilot
    "cam_iou_early": cam_iou_early_truncated,
    "cam_iou_late":  cam_iou_late_truncated,
    "cam_ins_early": cam_ins_early_truncated,
    "cam_ins_late":  cam_ins_late_truncated,
    "cam_del_early": cam_del_early_truncated,
    "cam_del_late":  cam_del_late_truncated,
    # Saliency pilot
    "sal_iou":        sal_iou_list_truncated,
    "sal_ins":        sal_ins_list_truncated,
    "sal_del":        sal_del_list_truncated,
    # IG pilot
    "ig_iou_early":   ig_iou_early_truncated,
    "ig_iou_late":    ig_iou_late_truncated,
    "ig_ins_early":   ig_ins_early_truncated,
    "ig_ins_late":    ig_ins_late_truncated,
    "ig_del_early":   ig_del_early_truncated,
    "ig_del_late":    ig_del_late_truncated,
})

# 4. Expand Saliency into early/late for consistency
df_pilot["sal_iou_early"] = df_pilot["sal_iou"]
df_pilot["sal_iou_late"]  = df_pilot["sal_iou"]
df_pilot["sal_ins_early"] = df_pilot["sal_ins"]
df_pilot["sal_ins_late"]  = df_pilot["sal_ins"]
df_pilot["sal_del_early"] = df_pilot["sal_del"]
df_pilot["sal_del_late"]  = df_pilot["sal_del"]

# 5. Drop the generic sal_* columns
df_pilot = df_pilot.drop(columns=["sal_iou","sal_ins","sal_del"])

# Assign to df as the original cell intended
df = df_pilot

# 6. Take a peek
display(df.head())

In [ ]:
df_pilot["lime_iou_early"] = lime_iou
df_pilot["lime_ins_early"] = lime_ins
df_pilot["lime_del_early"] = lime_del
df_pilot["lime_iou_late"]  = lime_iou
df_pilot["lime_ins_late"]  = lime_ins
df_pilot["lime_del_late"]  = lime_del


In [ ]:
# ─── Cell: Aggregate 400-image metrics into df_full ────────────────────────
import pandas as pd
import numpy as np

# 1) Sanity-check that all your 400-image lists exist and have the same length
n = len(gc_iou_e)
# Use the actual variable names for Saliency metrics on the full dataset
lists = [
    gc_iou_l, gc_ins_e, gc_ins_l, gc_del_e, gc_del_l,        # Grad-CAM
    sal_iou_list, sal_ins_list, sal_del_list,                # Saliency (using actual names)
    full_iou_e, full_iou_l, full_ins_e, full_ins_l, full_del_e, full_del_l  # IG
]
assert all(len(lst)==n for lst in lists), f"List length mismatch: expected {n}"

# 2) Build the DataFrame
df_full = pd.DataFrame({
    # Grad-CAM
    "cam_iou_early": gc_iou_e,
    "cam_iou_late":  gc_iou_l,
    "cam_ins_early": gc_ins_e,
    "cam_ins_late":  gc_ins_l,
    "cam_del_early": gc_del_e,
    "cam_del_late":  gc_del_l,
    # Saliency (same for early/late) - using actual names
    "sal_iou_early": sal_iou_list,
    "sal_iou_late":  sal_iou_list,
    "sal_ins_early": sal_ins_list,
    "sal_ins_late":  sal_ins_list,
    "sal_del_early": sal_del_list,
    "sal_del_late":  sal_del_list,
    # Integrated Gradients
    "ig_iou_early":  full_iou_e,
    "ig_iou_late":   full_iou_l,
    "ig_ins_early":  full_ins_e,
    "ig_ins_late":   full_ins_l,
    "ig_del_early":  full_del_e,
    "ig_del_late":   full_del_l,
})

# 3) Quick sanity print
print(f"df_full: {df_full.shape[0]} rows × {df_full.shape[1]} columns")
df_full.head()

In [ ]:
df_full["lime_iou_early"] = lime_full_iou
df_full["lime_ins_early"] = lime_full_ins
df_full["lime_del_early"] = lime_full_del
df_full["lime_iou_late"]  = lime_full_iou
df_full["lime_ins_late"]  = lime_full_ins
df_full["lime_del_late"]  = lime_full_del


In [ ]:
# ─── Cell 21.1: Attribution Stability under Gaussian Noise (400 images) ────
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torch

# 1) Settings
noise_sigmas = [0.01, 0.05, 0.1]    # relative noise std-dev
n_images     = 400                  # cap at 400 VOC images
methods      = ["Saliency", "Grad-CAM", "IG"]
stab_results = {m: [] for m in methods}

# 2) Helper to get any method’s map
# Assuming compute_saliency, compute_cam, compute_ig_map, model, preprocess, device are defined
def get_map(img_bgr, method):
    if method == "Saliency":
        # Assuming compute_saliency takes img_bgr
        return compute_saliency(img_bgr, model, preprocess, device)
    elif method == "Grad-CAM":
        # Replace compute_attribution with compute_cam
        # compute_cam expects img_bgr, model, preprocess, device, and optional target_layer
        # Assuming we use the default target_layer for this stability test
        heatmap, _ = compute_cam(img_bgr, model, preprocess, device=device)
        return heatmap # compute_cam returns heatmap and overlay, we need the heatmap
    elif method == "IG":
        # compute_ig_map expects a tensor and target_class
        # Need to convert img_bgr to tensor and get target_class
        img_pil = Image.fromarray(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
        inp = preprocess(img_pil).unsqueeze(0).to(device)
        with torch.no_grad():
            cls = int(model(inp).argmax(dim=1).item())
        # Assuming compute_ig_map takes input tensor, target_class, original_shape, n_steps
        return compute_ig_map(inp, cls, original_shape=img_bgr.shape[:2], n_steps=ig_steps_full)
    else:
        raise ValueError(f"Unknown method: {method}")

# 3) Stability loop
# Assuming dataset is available from previous cells and ig_steps_full is defined
start = time.time() # Define start
for sigma in noise_sigmas:
    temp = {m: [] for m in methods}
    # Iterate over the first n_images from the dataset
    for idx, (img_pil, _) in enumerate(dataset):
        if idx >= n_images: # Use n_images (400)
            break

        # PIL→RGB→BGR conversion is done inside get_map or should be done here?
        # Let's keep the original structure and convert before calling get_map
        if isinstance(img_pil, (str, Path)):
            img_pil = Image.open(img_pil).convert("RGB")
        elif isinstance(img_pil, Image.Image):
             img_pil = img_pil.convert("RGB")
        elif isinstance(img_pil, torch.Tensor):
            t = img_pil.detach().cpu()
            if t.ndim == 4 and t.size(0)==1:
                t = t.squeeze(0)
            arr = (t.permute(1,2,0).clamp(0,1) * 255).byte().numpy()
            img_pil = Image.fromarray(arr)
        else:
             print(f"Skipping unrecognized image type at index {idx}: {type(img_pil)}")
             continue

        rgb     = np.array(img_pil, dtype=np.uint8)
        img_bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

        for m in methods:
            # Ensure get_map returns a 2D heatmap for IoU calculation
            # compute_saliency returns 2D, compute_cam returns 2D heatmap (first element), compute_ig_map returns 2D
            try:
                orig_map  = get_map(img_bgr, m)
            except Exception as e:
                print(f"Error computing original map for method {m} on image {idx}: {e}")
                continue # Skip this image for this method

            # add Gaussian noise in [0,255]
            noise     = np.random.randn(*img_bgr.shape).astype(np.float32) * sigma * 255
            noisy_bgr = np.clip(img_bgr.astype(np.float32) + noise, 0, 255).astype(np.uint8)

            try:
                noisy_map = get_map(noisy_bgr, m)
            except Exception as e:
                 print(f"Error computing noisy map for method {m} on image {idx} with sigma {sigma}: {e}")
                 continue # Skip this image for this method


            # IoU(original vs noisy) - need a function that computes IoU for two heatmaps
            # Assuming iou_score function defined elsewhere is suitable for heatmaps
            # Need to ensure heatmaps are in the correct format (e.g., binary masks or similar for iou_score)
            # The original code used iou_score on masks, so we might need to threshold the heatmaps here
            # Thresholding with percentile as done in other cells (e.g., FXdk--iP1-oW)
            # Using a fixed percentile for simplicity in stability test, e.g., 80 as used elsewhere
            best_p_stability = 80 # Using a threshold for binarization for IoU

            if orig_map is not None and noisy_map is not None and orig_map.shape == noisy_map.shape:
                try:
                    # Threshold heatmaps to create binary masks for IoU
                    thr_orig = np.percentile(orig_map, best_p_stability)
                    mask_orig = (orig_map >= thr_orig).astype(np.uint8)

                    thr_noisy = np.percentile(noisy_map, best_p_stability)
                    mask_noisy = (noisy_map >= thr_noisy).astype(np.uint8)

                    # Assuming iou_score takes two binary masks
                    temp[m].append(iou_score(mask_noisy, mask_orig))
                except Exception as e:
                     print(f"Error computing IoU for method {m} on image {idx} with sigma {sigma}: {e}")
            else:
                print(f"Skipping IoU for method {m} on image {idx} with sigma {sigma} due to invalid map shapes or None maps.")


    # record mean IoU for this sigma
    for m in methods:
        if temp[m]: # Only calculate mean if there are valid IoU scores
            stab_results[m].append(np.mean(temp[m]))
        else:
            stab_results[m].append(np.nan) # Append NaN if no valid scores for this sigma and method

# 4) Plot
plt.figure(figsize=(6,4), dpi=150)
for m in methods:
    # Only plot if there are valid results for this method
    if stab_results[m]:
        plt.plot(noise_sigmas[:len(stab_results[m])], stab_results[m], marker="o", label=m) # Ensure noise_sigmas matches results length

plt.xlabel("Input noise σ")
plt.ylabel("Mean IoU (orig vs noisy mask)")
plt.ylim(0,1)
plt.title("Attribution Stability under Gaussian Noise (400 images)")
plt.legend()
plt.show()

elapsed = time.time() - start # Calculate elapsed time after the loop and before plotting
print(f"\nStability analysis on {n_images} images done in {elapsed:.1f}s")

## Cell E: Plot Comparative Bar Charts for Attribution Metrics

This cell creates three side‐by‐side bar charts—one each for IoU, Insertion AUC, and Deletion AUC—to visually compare the three attribution methods (Grad-CAM, Saliency, and Integrated Gradients) over the first 200 VOC validation images.  

- **Labels**: `["CAM", "Saliency", "IG"]`  
- **Metric Groups**:  
  - `("cam_iou",  "sal_iou",  "ig_iou")`   → Intersection-over-Union scores  
  - `("cam_ins", "sal_ins", "ig_ins")`   → Insertion AUC (higher = better)  
  - `("cam_del", "sal_del", "ig_del")`   → Deletion AUC (higher = better)  

For each subplot:  
1. Compute the mean and standard deviation of the three methods for that metric.  
2. Draw a bar chart with error bars (± 1 std) and set y-axis limits to [0, 1].  
3. Title each subplot accordingly (“IoU”, “Ins AUC”, “Del AUC”).  

Finally, the figure is given an overall title (“Comparison of Attribution Methods (200 VOC images)”), and the DataFrame and figure can be optionally saved to disk.  


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = ["CAM", "Saliency", "IG", "LIME"]
x = np.arange(len(labels))
width = 0.35

metrics = ["iou", "ins", "del"]
titles  = ["IoU", "Ins AUC", "Del AUC"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5), dpi=150)

for ax, metric, title in zip(axes, metrics, titles):
    # Include LIME columns in the lists for calculating means and errors
    early_cols = [f"cam_{metric}_early", f"sal_{metric}_early", f"ig_{metric}_early", f"lime_{metric}_early"]
    late_cols  = [f"cam_{metric}_late",  f"sal_{metric}_late",  f"ig_{metric}_late",  f"lime_{metric}_late"]

    # Ensure df is defined and contains these columns. Assuming df is df_pilot or df_full
    # based on previous cells. Using df directly as it was the last assigned DataFrame.
    means_early = df[early_cols].mean().values
    errs_early  = df[early_cols].std().values
    means_late  = df[late_cols].mean().values
    errs_late   = df[late_cols].std().values

    ax.bar(x - width/2, means_early, width, yerr=errs_early, capsize=5, label="Early")
    ax.bar(x + width/2, means_late,  width, yerr=errs_late,  capsize=5, label="Late")

    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_ylim(0, 1.0)
    ax.set_title(title)
    if metric == "iou":
        ax.set_ylabel("Score")
    ax.legend()

plt.suptitle("Early vs. Late Layer Comparison of Attribution Metrics", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Common labels
methods = ["CAM", "Saliency", "IG"]
x = np.arange(len(methods))
width = 0.35

# Define the two runs and their DataFrames
runs = [
    ("Pilot (200 images)", df_pilot),
    ("Full  (400 images)", df_full)
]

metrics = [
    ("iou", "IoU"),
    ("ins", "Insertion AUC"),
    ("del", "Deletion AUC")
]

fig, axes = plt.subplots(
    nrows=len(runs), ncols=len(metrics),
    figsize=(15, 8), dpi=150
)

for i, (run_label, df_run) in enumerate(runs):
    for j, (metric, title) in enumerate(metrics):
        ax = axes[i, j]

        # collect early/late columns
        early_cols = [f"cam_{metric}_early", f"sal_{metric}_early", f"ig_{metric}_early"]
        late_cols  = [f"cam_{metric}_late",  f"sal_{metric}_late",  f"ig_{metric}_late"]

        means_early = df_run[early_cols].mean().values
        errs_early  = df_run[early_cols].std().values
        means_late  = df_run[late_cols].mean().values
        errs_late   = df_run[late_cols].std().values

        ax.bar(x - width/2, means_early, width, yerr=errs_early, capsize=5, label="Early")
        ax.bar(x + width/2, means_late,  width, yerr=errs_late,  capsize=5, label="Late")

        ax.set_xticks(x)
        ax.set_xticklabels(methods)
        ax.set_ylim(0, 1.0)
        ax.set_title(f"{title}\n{run_label}")
        if j == 0:
            ax.set_ylabel("Score")
        # only add legend once
        if i == 0 and j == len(metrics) - 1:
            ax.legend(loc="upper right")

plt.suptitle("Attribution Metrics: Pilot vs. Full Runs", y=1.04)
plt.tight_layout()
plt.show()


## Cell D: Build 5×4 “Survey Examples” Grid

This cell selects five representative VOC validation images (by index), runs the model to predict each image’s top class, draws the ground‐truth bounding boxes, and then visualizes all three attribution methods side‐by‐side. Specifically:  
1. **Choose indices** (`idx_list = [10, 88, 122, 450, 1200]`).  
2. **Loop** over each index:  
   - Load the PIL image and its GT boxes.  
   - Convert to BGR for OpenCV, preprocess for ResNet50, and run a forward pass to get the top predicted class name.  
   - Overlay GT bounding boxes in red on a copy of the original.  
   - Compute the three attribution heatmaps (Grad-CAM, Saliency, IG), resize them back to the original image size, and overlay each on the RGB image using a semi-transparent “jet” colormap.  
3. **Arrange** these five images in a 5-row × 4-column grid:  
   - Column 1: “Original + GT boxes” with the predicted class label.  
   - Column 2: Grad-CAM overlay.  
   - Column 3: Saliency overlay.  
   - Column 4: Integrated Gradients overlay.  
4. **Add a super‐title** (“Survey Examples: Original / Grad-CAM / Saliency / IG (5 Images)”).  
5. **Save** the resulting figure as `survey_examples_with_labels_and_boxes.png` for use in the questionnaire.  


In [ ]:
# ── Cell S1: Survey Stage 1 – Overlays Only (group by example) ─────────────
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

idx_list = [10, 88, 122, 450, 1200]
late_layer = model.layer4[-1]
n_rows, n_cols = len(idx_list), 4
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows), dpi=100)

# Letters A–T for 5×4 grid
letters = [chr(ord('A') + i) for i in range(n_rows * n_cols)]
titles  = ["Original+GT", "CAM Late", "Saliency", "IG Late"]

for r, img_idx in enumerate(idx_list):
    # Load and draw GT
    img_pil, gt_boxes = dataset[img_idx]
    img_rgb = np.array(img_pil.convert("RGB"))
    viz = img_rgb.copy()
    for (x1,y1,x2,y2) in gt_boxes:
        cv2.rectangle(viz, (x1,y1), (x2,y2), (255,0,0), 2)

    # Compute overlays
    img_bgr  = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cam_l, _ = compute_cam(img_bgr, model, preprocess,
                           target_layer=late_layer, device=device)
    sal_map  = compute_saliency(img_bgr, model, preprocess, device)
    ig_l     = compute_ig(img_bgr, model, preprocess, device,
                          n_steps=20, target_layer=late_layer)

    overlays = [viz, cam_l, sal_map, ig_l]

    for c in range(n_cols):
        ax = axes[r, c]
        # Letter annotation
        ax.text(0.02, 0.90, letters[r*n_cols + c],
                transform=ax.transAxes,
                fontsize=12, fontweight='bold', color='white')
        if c == 0:
            im = ax.imshow(overlays[c], vmin=0, vmax=255)
        else:
            ax.imshow(viz, vmin=0, vmax=255)
            im = ax.imshow(overlays[c], cmap='jet', alpha=0.6,
                           vmin=0, vmax=1)
        ax.set_title(f"{titles[c]}", fontsize=10)
        ax.axis('off')

# Shared colorbar for columns 1–3
cbar = fig.colorbar(im, ax=axes[:,1:], orientation='vertical',
                    fraction=0.02, pad=0.01, label='Attribution intensity')

plt.tight_layout()
plt.subplots_adjust(top=0.95, hspace=0.3)
plt.suptitle("Stage 1: Overlays Only (Late-Layer Comparisons)", fontsize=14)
plt.show()

fig.savefig("survey_stage1_overlays.png", dpi=200, bbox_inches="tight")
print("✅ Saved survey_stage1_overlays.png")


In [ ]:
# ── Cell S1: Survey Stage 1 – Overlays Only (group by example, with LIME) ──
import matplotlib.pyplot as plt, numpy as np, cv2
from PIL import Image

idx_list   = [10, 88, 122, 450, 1200]
late_layer = model.layer4[-1]
n_rows, n_cols = len(idx_list), 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4 * n_rows), dpi=100)

letters = [chr(ord('A') + i) for i in range(n_rows * n_cols)]
titles  = ["Original+GT", "CAM Late", "Saliency", "IG Late", "LIME"]

for r, img_idx in enumerate(idx_list):
    # ── 0) load image & GT boxes ───────────────────────────────────────────
    img_pil, gt_boxes = dataset[img_idx]
    img_rgb = np.array(img_pil.convert("RGB"))
    viz     = img_rgb.copy()
    for (x1, y1, x2, y2) in gt_boxes:
        cv2.rectangle(viz, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # ── 0.1) predict class & name for annotation ──────────────────────────
    with torch.no_grad():
        cls_idx  = int(model(preprocess(img_pil).unsqueeze(0).to(device)).argmax(1))
        cls_name = weights.meta["categories"][cls_idx]

    # ── 1) compute maps ───────────────────────────────────────────────────
    img_bgr  = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cam_l, _ = compute_cam(img_bgr, model, preprocess, target_layer=late_layer, device=device)
    sal_map  = compute_saliency(img_bgr, model, preprocess, device)
    ig_l     = compute_ig(img_bgr, model, preprocess, device, n_steps=20, target_layer=late_layer)
    lime_map = compute_lime_map(img_bgr, model, preprocess, device)

    overlays = [viz, cam_l, sal_map, ig_l, lime_map]

    # ── 2) plot each column ───────────────────────────────────────────────
    for c in range(n_cols):
        ax = axes[r, c]
        ax.text(0.02, 0.90, letters[r*n_cols + c],
                transform=ax.transAxes, fontsize=12, fontweight='bold', color='white')

        if c == 0:                                          # original + GT column
            ax.imshow(overlays[c], vmin=0, vmax=255)
            ax.set_title(f"{titles[c]}\nPred: {cls_name}", fontsize=10)
        else:                                               # attribution overlays
            ax.imshow(viz, vmin=0, vmax=255)
            im = ax.imshow(overlays[c], cmap='jet', alpha=0.5, vmin=0, vmax=1)
            ax.set_title(titles[c], fontsize=10)
        ax.axis('off')

# shared color-bar and layout tweaks (unchanged)
cbar = fig.colorbar(im, ax=axes[:,1:], orientation='vertical',
                    fraction=0.02, pad=0.01, label='Attribution intensity')

plt.tight_layout()
plt.subplots_adjust(top=0.95, hspace=0.3)
plt.suptitle("Stage 1: Overlays Only (Late-Layer + LIME)", fontsize=14)
plt.show()

fig.savefig("survey_stage1_overlays_lime.png", dpi=200, bbox_inches="tight")
print("✅ Saved survey_stage1_overlays_lime.png")


In [ ]:
# ── Cell S2′: Survey Stage 2 – Originals + Early vs. Late (CAM & IG) ─────────────
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

idx_list    = [10, 88, 122, 450, 1200]
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]
methods     = [("CAM", compute_cam), ("IG", compute_ig)]

n_rows = len(idx_list)
n_cols = 1 + len(methods)*2   # 1 for original, 2 layers × 2 methods = 5
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(18, 4 * n_rows),
                         dpi=100)

# Generate letter annotations A, B, C...
total_panels = n_rows * n_cols
letters = [chr(ord('A') + i) for i in range(total_panels)]

for r, img_idx in enumerate(idx_list):
    # 0) Load image + GT boxes
    img_pil, gt_boxes = dataset[img_idx]
    img_rgb = np.array(img_pil.convert("RGB"))
    viz     = img_rgb.copy()
    for x1,y1,x2,y2 in gt_boxes:
        cv2.rectangle(viz, (x1,y1), (x2,y2), (255,0,0), 2)

    # 1) Predict class (optional if you want title)
    inp = preprocess(img_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        cls_idx  = int(model(inp).argmax(dim=1).item())
        cls_name = weights.meta["categories"][cls_idx]

    # 2) Compute heatmaps
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cams    = []
    igs     = []
    for name, fn in methods:
        if name == "CAM":
            cam_e, _ = fn(img_bgr, model, preprocess,
                          target_layer=early_layer, device=device)
            cam_l, _ = fn(img_bgr, model, preprocess,
                          target_layer=late_layer,  device=device)
            cams.extend([cam_e, cam_l])
        else:  # IG
            ig_e = fn(img_bgr, model, preprocess, device,
                      baseline=None, n_steps=20,
                      target_layer=early_layer)
            ig_l = fn(img_bgr, model, preprocess, device,
                      baseline=None, n_steps=20,
                      target_layer=late_layer)
            igs.extend([ig_e, ig_l])

    heatmaps = cams + igs  # order: CAM early, CAM late, IG early, IG late

    # 3) Plot original + GT in col 0
    ax = axes[r, 0]
    idx_letter = r*n_cols + 0
    ax.text(0.02, 0.90, letters[idx_letter],
            transform=ax.transAxes,
            fontsize=12, fontweight='bold', color='white')
    ax.imshow(viz, vmin=0, vmax=255)
    ax.set_title(f"Original + GT\nIdx {img_idx}", fontsize=10)
    ax.axis('off')

    # 4) Plot heatmap columns
    for c in range(1, n_cols):
        ax = axes[r, c]
        idx_letter = r*n_cols + c
        ax.text(0.02, 0.90, letters[idx_letter],
                transform=ax.transAxes,
                fontsize=12, fontweight='bold', color='white')

        hm = heatmaps[c-1]
        im = ax.imshow(hm, cmap='jet', vmin=0, vmax=1)
        method_idx = (c-1) // 2
        layer_lbl  = "Early" if (c-1) % 2 == 0 else "Late"
        ax.set_title(f"{methods[method_idx][0]} {layer_lbl}", fontsize=10)
        ax.axis('off')

# 5) Shared colorbar for heatmaps
cbar = fig.colorbar(im, ax=axes[:, 1:], orientation='vertical',
                    fraction=0.02, pad=0.01, label='Attribution intensity')

plt.tight_layout()
plt.subplots_adjust(top=0.94, hspace=0.3)
plt.suptitle("Stage 2: Originals + CAM/IG Early vs. Late", fontsize=14)
plt.show()

fig.savefig("survey_stage2_with_originals.png", dpi=200, bbox_inches="tight")
print("✅ Saved as survey_stage2_with_originals.png")


In [ ]:
# ── Cell S2: Survey Stage 2 – Originals + CAM, IG & LIME (standalone heatmaps) ──
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

idx_list    = [10, 88, 122, 450, 1200]
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]
methods     = [("CAM", compute_cam), ("IG", compute_ig), ("LIME", compute_lime_map)]

n_rows = len(idx_list)
n_cols = 1 + 2 + 2 + 1   # 1 original, 2 CAM, 2 IG, 1 LIME = 6
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(18, 4 * n_rows),
                         dpi=100)

# Letter labels A–Z…
letters = [chr(ord('A') + i) for i in range(n_rows * n_cols)]
titles  = ["Original+GT",
           "CAM Early", "CAM Late",
           "IG Early",  "IG Late",
           "LIME"]

for r, img_idx in enumerate(idx_list):
    img_pil, gt_boxes = dataset[img_idx]
    img_rgb = np.array(img_pil.convert("RGB"))
    viz     = img_rgb.copy()
    for (x1,y1,x2,y2) in gt_boxes:
        cv2.rectangle(viz, (x1,y1), (x2,y2), (255,0,0), 2)

    # 1) Original + GT
    ax = axes[r, 0]
    ax.text(0.02, 0.90, letters[r*n_cols + 0],
            transform=ax.transAxes,
            fontsize=12, fontweight='bold', color='white')
    ax.imshow(viz, vmin=0, vmax=255)
    ax.set_title(titles[0], fontsize=10)
    ax.axis('off')

    # 2) Compute and plot each attribution map
    heatmaps = []
    # 2a) CAM
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cam_e, _ = compute_cam(img_bgr, model, preprocess,
                          target_layer=early_layer, device=device)
    cam_l, _ = compute_cam(img_bgr, model, preprocess,
                          target_layer=late_layer,  device=device)
    heatmaps += [cam_e, cam_l]
    # 2b) IG
    ig_e = compute_ig(img_bgr, model, preprocess,
                      device=device, baseline=None,
                      n_steps=20, target_layer=early_layer)
    ig_l = compute_ig(img_bgr, model, preprocess,
                      device=device, baseline=None,
                      n_steps=20, target_layer=late_layer)
    heatmaps += [ig_e, ig_l]
    # 2c) LIME
    lime_map = compute_lime_map(img_bgr, model, preprocess, device)
    heatmaps.append(lime_map)

    # 3) Plot heatmaps in columns 1..5
    for c in range(1, n_cols):
        ax = axes[r, c]
        idx_letter = r*n_cols + c
        ax.text(0.02, 0.90, letters[idx_letter],
                transform=ax.transAxes,
                fontsize=12, fontweight='bold', color='white')

        hm = heatmaps[c-1]
        im = ax.imshow(hm, cmap='jet', vmin=0, vmax=1)
        ax.set_title(titles[c], fontsize=10)
        ax.axis('off')

# 4) Shared colorbar for all attribution columns
cbar = fig.colorbar(im, ax=axes[:,1:], orientation='vertical',
                    fraction=0.02, pad=0.01, label='Attribution intensity')

plt.tight_layout()
plt.subplots_adjust(top=0.94, hspace=0.3)
plt.suptitle("Stage 2: Originals + Standalone Heatmaps (CAM, IG, LIME)", fontsize=14)
plt.show()

fig.savefig("survey_stage2_standalone_heatmaps.png", dpi=200, bbox_inches="tight")
print("✅ Saved survey_stage2_standalone_heatmaps.png")


In [ ]:
# ── Cell: 10×6 Survey Examples with Overlays + Standalone Heatmaps (using compute_ig) ─────────────
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import torch

# 1) Pick five VOC‐validation indices:
idx_list = [10, 88, 122, 450, 1200]

# 2) Early vs. late layers:
early_layer = model.layer1[0]
late_layer  = model.layer4[-1]

# 3) Build a figure: 10 rows (5 examples × 2) × 6 columns
n_examples = len(idx_list)
n_rows     = n_examples * 2
n_cols     = 6
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(20, 2 * n_rows),
                         dpi=100)

# 4) Loop through each example
for i, img_idx in enumerate(idx_list):
    top = 2 * i      # overlay row
    bot = top + 1    # heatmap row

    # 4a) Load image & GT
    img_pil, gt_boxes = dataset[img_idx]
    img_rgb = np.array(img_pil.convert("RGB"))
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    H, W    = img_bgr.shape[:2]

    # 4b) Predict class
    inp_tensor = preprocess(img_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        logits  = model(inp_tensor)
        cls_idx = int(logits.argmax(dim=1).item())
        cls_name= weights.meta["categories"][cls_idx]

    # 4c) Draw ground-truth boxes
    viz = img_rgb.copy()
    for x1, y1, x2, y2 in gt_boxes:
        cv2.rectangle(viz, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # 4d) Compute Grad-CAM at early & late
    cam_e, _ = compute_cam(img_bgr, model, preprocess,
                           target_layer=early_layer, device=device)
    cam_l, _ = compute_cam(img_bgr, model, preprocess,
                           target_layer=late_layer, device=device)

    # 4e) Compute Saliency
    sal_map  = compute_saliency(img_bgr, model, preprocess, device)

    # 4f) Compute IG at early & late via compute_ig()
    ig_e = compute_ig(img_bgr, model, preprocess, device,
                      baseline=None, n_steps=20,
                      target_layer=early_layer)
    ig_l = compute_ig(img_bgr, model, preprocess, device,
                      baseline=None, n_steps=20,
                      target_layer=late_layer)

    # 4g) Prepare overlays and titles
    overlays = [viz, cam_e, cam_l, sal_map, ig_e, ig_l]
    titles   = [
        f"Idx {img_idx}\nPred: {cls_name}",
        "CAM Early", "CAM Late",
        "Saliency", "IG Early", "IG Late"
    ]

    # --- Overlay row (top) ---
    for c in range(n_cols):
        ax = axes[top, c]
        if c == 0:
            ax.imshow(overlays[c])
        else:
            ax.imshow(viz)
            ax.imshow(overlays[c], cmap="jet", alpha=0.5)
        ax.set_title(titles[c], fontsize=10)
        ax.axis("off")

    # --- Heatmap row (bot) ---
    for c in range(n_cols):
        ax = axes[bot, c]
        if c == 0:
            ax.axis("off")
        else:
            ax.imshow(overlays[c], cmap="jet")
            ax.set_title("Heatmap", fontsize=8)
            ax.axis("off")

# 5) Layout adjustments and save
plt.tight_layout()
plt.subplots_adjust(top=0.95, hspace=0.2)
plt.suptitle("Survey: Overlays (odd rows) + Heatmaps (even rows)", fontsize=14)
plt.show()

fig.savefig("survey_examples_with_heatmaps.png", dpi=200, bbox_inches="tight")
print("✅ Saved as survey_examples_with_heatmaps.png")


### Cell: Visualize CAM Results

Visualize the CAM heatmaps and their overlays on the original image for both the early and late layers of the ResNet model.

### Cell: Visualize CAM Results

Visualize the CAM heatmaps and their overlays on the original image for both the early and late layers of the ResNet model.